In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models

from Dataset import MyCOCO
from Train import train_model

In [3]:
# Load the datasets

datasets = {x: MyCOCO(mode = x) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size = 32, shuffle = True, num_workers = 4) for x in ['train', 'val']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}

loading annotations into memory...
Done (t=9.45s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


In [4]:
model_choice = 1

if model_choice == 0:
    model = models.resnet18(pretrained = True)
elif model_choice == 1:
    model = models.mobilenet_v2(pretrained = True)

for param in model.parameters():
    param.requires_grad = False

if model_choice == 0:
    model.fc = torch.nn.Linear(in_features = 512, out_features = 91)
    optim_params = model.fc.parameters()
elif model_choice == 1:
    model.classifier[1] = torch.nn.Linear(in_features = 1280, out_features = 91)
    optim_params = model.classifier.parameters()
    
print(model)


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [5]:

model.cuda()

criterion = torch.nn.BCEWithLogitsLoss()

optimizer = optim.Adam(optim_params, lr= 0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 2, gamma = 0.1)

model = train_model(model, dataloaders, dataset_sizes, criterion, optimizer, exp_lr_scheduler, num_epochs = 5)

torch.save(model.state_dict(), './model_transfer.pt')


  0%|          | 0/3697 [00:00<?, ?it/s]

Epoch 0/4
----------


  0%|          | 0/157 [00:00<?, ?it/s]

train Loss: 0.0747 Acc: 0.9678


  0%|          | 0/3697 [00:00<?, ?it/s]

...

val Loss: 0.0652 Acc: 0.9686

Epoch 4/4
----------


  0%|          | 0/157 [00:00<?, ?it/s]

train Loss: 0.0663 Acc: 0.9686


100%|██████████| 157/157 [00:09<00:00, 16.57it/s]

val Loss: 0.0645 Acc: 0.9684

Training complete in 19m 31s
Best val Acc: 0.968600


In [6]:
model.eval()

y_hat = []
y_true = []

for inputs, labels in dataloaders['val']:
    
    y_hat.append(1.0 * (model(inputs.cuda()).cpu().data.numpy() > 0.0))
        
    y_true.append(labels.numpy())
        
y_hat = np.concatenate(np.array(y_hat), axis = 0)
y_true = np.concatenate(np.array(y_true), axis = 0)

dim = y_hat.shape[1]

precision = np.zeros((dim))

for i in range(dim):
    precision[i] = precision_score(np.squeeze(y_true[:, i]), np.squeeze(y_hat[:, i]), zero_division = 0)
    
print("Precision:", precision)

MAP = 0
MAP_count = 0
for p in precision:
    if p != 0.0: # WARNING:  this is meant to ignore the unused indices of the labels, but may inflate the results
        MAP += p
        MAP_count += 1

print("MAP:", MAP/ MAP_count)

Precision: [0.         0.90137987 0.71794872 0.6959707  0.80808081 0.81081081
 0.79464286 0.87850467 0.62765957 0.73134328 0.69473684 0.89655172
 0.         0.8        0.66666667 0.65714286 0.96551724 0.86507937
 0.81818182 0.8        0.86363636 0.875      0.8902439  0.91428571
 0.91566265 0.90625    0.         0.5        0.81818182 0.
 0.         0.62745098 0.75903614 0.72222222 0.69047619 0.83333333
 0.54545455 0.55633803 0.73214286 0.7037037  0.86419753 0.73684211
 0.87931034 0.92537313 0.5915493  0.         0.5        0.60493827
 0.5        0.48275862 0.47916667 0.63157895 0.8        0.85714286
 0.62222222 0.73170732 0.74242424 0.63888889 0.71428571 0.86956522
 0.54545455 0.7        0.69230769 0.68539326 0.65217391 0.80769231
 0.         0.71821306 0.         0.         0.80314961 0.
 0.73728814 0.79518072 0.765625   0.61764706 0.78787879 0.7
 0.59090909 0.67307692 0.         0.81818182 0.75       0.
 0.63157895 0.82653061 0.76923077 0.66666667 0.88888889 0.
 0.66666667]
MAP: 0.736